<a href="https://colab.research.google.com/github/russell-ai/topic-covid/blob/main/tweet_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis on Twitter


# load data

In [1]:
#@title Drive Downloader

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

download_with_pydrive = True #@param {type:"boolean"}  

class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()
        
    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_id, file_dst):
        if self.use_pydrive:
            downloaded = self.drive.CreateFile({'id':file_id})
            downloaded.FetchMetadata(fetch_all=True)
            downloaded.GetContentFile(file_dst)
        else:
            !gdown --id file_dst

downloader = Downloader(download_with_pydrive)

In [ ]:
#https://drive.google.com/file/d/1sjtke6Y_3wMlBQoYCHoANQu7Fj7l7rrn/view?usp=share_link

In [2]:
current_directory = os.getcwd()
save_path = os.path.join(os.path.dirname(current_directory), "data")
os.makedirs(save_path, exist_ok=True)
model_path = os.path.join(save_path, 'tweet_data.csv')

In [3]:
downloader.download_file("1sjtke6Y_3wMlBQoYCHoANQu7Fj7l7rrn", file_dst=model_path)

### fetching data as dataframe

In [1]:
import pandas as pd
df = pd.read_csv("/data/tweet_data.csv")
df.head()

,Tweet_Id,Tweet
0,1.598104e+18,@elonmusk I thunk you're an irresponsible for ...
1,1.598104e+18,"Apparently ""I've had COVID for over a week and..."
2,1.598103e+18,@snakewasright @EllenOl00313689 @iTruthSearch ...
3,1.598103e+18,They're dropping the sequel to COVID baby wooo...
4,1.598101e+18,@1goodtern They drop dead one by one of post C...


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446216 entries, 0 to 446215
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Tweet_Id  446216 non-null  float64
 1   Tweet     446216 non-null  object 
dtypes: float64(1), object(1)
memory usage: 6.8+ MB


# Setting up and loading model

In [6]:
!pip install -q transformers

     |████████████████████████████████| 5.8 MB 7.0 MB/s 
     |████████████████████████████████| 7.6 MB 72.2 MB/s 
     |████████████████████████████████| 182 kB 70.3 MB/s 


In [3]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
hf_token = open("/content/drive/MyDrive/token.txt").read()


In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True)

model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Setting pipeline for prediction

In [21]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [22]:
from transformers import pipeline
sentiment_task = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device="cuda:0")
sentiment_task("Covid cases are increasing fast!")


[{'label': 'negative', 'score': 0.7235767841339111}]

# feeding pipeline with dataset

### faster with swifter

In [ ]:
#! pip install swifter # first time installation
#! pip install swifter[groupby] # first time installation including dependency for groupby.apply functionality

In [23]:
import swifter
import numpy as np

In [24]:
def sent_analysis(s):
  try:
    result = sentiment_task(s)
    label = result[0]["label"]
    score = result[0]["score"]
  except:
      label = np.nan
      score = np.nan
  return label,score


## prediction time

In [ ]:
df["sentiment_analysis"] = df["Tweet"].swifter.apply(lambda x: sent_analysis(x))

Pandas Apply:   0%|          | 0/446216 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [17]:
df.head()

,Tweet_Id,Tweet
0,1.598104e+18,@elonmusk I thunk you're an irresponsible for ...
1,1.598104e+18,"Apparently ""I've had COVID for over a week and..."
2,1.598103e+18,@snakewasright @EllenOl00313689 @iTruthSearch ...
3,1.598103e+18,They're dropping the sequel to COVID baby wooo...
4,1.598101e+18,@1goodtern They drop dead one by one of post C...


In [61]:
df["sentiment"] = df["sentiment_analysis"].swifter.apply(lambda x:x[0])

In [ ]:
df["score"] = df["sentiment_analysis"].swifter.apply(lambda x:x[1])

In [62]:
df.head(20)

,Tweet_Id,Tweet,sentiment,"(sentiment, x)",sentiment_analysis,score
0,1.598104e+18,@elonmusk I thunk you're an irresponsible for ...,negative,"(negative, 0.8987942934036255)","(negative, 0.8987942934036255)",0.898794
1,1.598104e+18,"Apparently ""I've had COVID for over a week and...",negative,"(negative, 0.5844964981079102)","(negative, 0.5844964981079102)",0.584496
2,1.598103e+18,@snakewasright @EllenOl00313689 @iTruthSearch ...,negative,"(negative, 0.7049753665924072)","(negative, 0.7049753665924072)",0.704975
3,1.598103e+18,They're dropping the sequel to COVID baby wooo...,neutral,"(neutral, 0.5086729526519775)","(neutral, 0.5086729526519775)",0.508673
4,1.598101e+18,@1goodtern They drop dead one by one of post C...,negative,"(negative, 0.4738270342350006)","(negative, 0.4738270342350006)",0.473827


### count the number of tweets

In [ ]:
sentiment_counts = df.groupby(['sentiment']).size()
print(sentiment_counts)


### visualize each sentiment

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(6,6), dpi=100)
ax = plt.subplot(111)
sentiment_counts.plot.pie(ax=ax, autopct='%1.1f%%', startangle=270, fontsize=12, label="")


In [ ]:
from wordcloud import WordCloud
from wordcloud import STOPWORDS
 
# Wordcloud with positive tweets
positive_tweets = df['tweet'][df["sentiment"] == 'Positive']
stop_words = ["https", "co", "RT"] + list(STOPWORDS)
positive_wordcloud = WordCloud(max_font_size=50, max_words=50, background_color="white", stopwords = stop_words).generate(str(positive_tweets))
plt.figure()
plt.title("Positive Tweets - Wordcloud")
plt.imshow(positive_wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
 

In [ ]:
# Wordcloud with negative tweets
negative_tweets = df['tweet'][df["sentiment"] == 'Negative']
stop_words = ["https", "co", "RT"] + list(STOPWORDS)
negative_wordcloud = WordCloud(max_font_size=50, max_words=50, background_color="white", stopwords = stop_words).generate(str(negative_tweets))
plt.figure()
plt.title("Negative Tweets - Wordcloud")
plt.imshow(negative_wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

## saving analysed dataset

In [63]:
df.to_csv("/content/drive/MyDrive/Colab Notebooks/PROJECTS/TOPIC-COVID/sent_tweet_data.csv", index=False)

source for [sentiment modelling  ](https://huggingface.co/blog/sentiment-analysis-twitter)


